# Construction de H

## Ancien code

In [1]:
import pandas as pd
import numpy as np

class CarboneAlpha:
    """Classe pour représenter un carbone alpha d'une protéine."""
    
    def __init__(self, number, x, y, z):
        self.number = number
        self.x = x
        self.y = y
        self.z = z
    
    def compute_distance(self, other):
        dist = ((other.x - self.x) ** 2 + (other.y - self.y) ** 2 + (other.z - self.z) ** 2) ** 0.5
        return dist

class Template:
    """Classe pour représenter le template utilisé."""
    
    def __init__(self, file):
        self.structure = self.build_template_from_pdb(file)
        self.length = len(self.structure)

    def build_template_from_pdb(self, filename):
        list_calpha = []
        with open(filename, "r") as pdb :
            for ligne in pdb:
                if ligne.startswith("ATOM") and (ligne[12:16].strip() == "CA"):
                    number = ligne[6:11].strip()
                    x = float(ligne[30:38].strip())
                    y = float(ligne[38:46].strip())
                    z = float(ligne[46:54].strip())
                                       
                    list_calpha.append(CarboneAlpha(number, x, y, z))
        return list_calpha
        
    def build_dist_matrix(self):
        dist_list = []
        
        for i, atom in enumerate(self.structure):
            dist_ligne = []
            for other in (self.structure):
                dist_ligne.append(atom.compute_distance(other))
            dist_list.append(dist_ligne)
            
        dist_matrix = np.array(dist_list)
        return dist_matrix
    
    def __str__(self):
        string = ""
        for i, ca in enumerate(self.structure):
            string += f"position {i}-{ca.number}, coor( {ca.x}, {ca.y}, {ca.z})\n"
        return string

def clean_DOPE_data(filename):
    ca_matrix = []
    
    with open(filename, "r") as dope :
        for ligne in dope:
            if ligne[3:7].strip() == "CA" and ligne[11:14].strip() == "CA":
                ca_matrix.append(ligne.split())
    
    columns = ['res1', 'temp1', 'res2', 'temp2'] + list(np.arange(0.25, 15, 0.5))
    dope_score = pd.DataFrame(ca_matrix, columns = columns) 
    dope_score = dope_score.drop(['temp1', 'temp2'], axis=1)
    
    return dope_score

class DynamicMatrix:    
    def __init__(self, lines, columns, gap):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.gap = gap

    def initialize_matrix(self, first_val, start, end, get_score):
        if (start[0] < 0) or (start[1] < 0):
            raise ValueError("Start of initialization out of matrix.")
        if (end[0] >= self.lines) or (end[1] >= self.columns):
            raise ValueError("End of initialization out of matrix.")
        
        # Première case
        self.matrix[start[0], start[1]] = first_val
        
        # Remplissage de la première colonne jusqu'à la limite
        for i in range(start[0] + 1, end[0] + 1):
            self.matrix[i, start[1]] = self.matrix[i - 1, start[1]] + self.gap + get_score(i, start[1])

        # Remplissage de la première ligne jusqu'à la limite
        for j in range(start[1] + 1, end[1] + 1):
            self.matrix[start[0], j] = self.matrix[start[0], j - 1] + self.gap + get_score(start[0], j)

class LowLevelMatrix(DynamicMatrix):
    aa_codes = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
    }
    
    def __init__(self, gap, frozen, distance, dope, sequence):
        lines = len(sequence)
        columns = len(distance)
        
        DynamicMatrix.__init__(self, lines, columns, gap)

        # Vérification du blocage de la case
        if (frozen['seq_id'] >= lines) or (frozen['seq_id'] < 0):
            raise ValueError("Frozen line index out of matrix.")
        if (frozen['pos_id'] >= columns) or (frozen['pos_id'] < 0):
            raise ValueError("Frozen column index out of matrix")

        # Récupération du résidu fixé
        frozen['seq_res'] = sequence[frozen['seq_id']]
        
        self.frozen = frozen
        self.distance = distance
        self.dope = dope
        self.sequence = sequence

    def round_distance(self, dist):
        # arrondi au quart le plus proche
        rounded_value = round(dist * 4) / 4
        
        # ne garde que 0.25 ou 0.75
        decimal = rounded_value % 1
        if decimal == 0.0:
            return rounded_value + 0.25
        elif decimal == 0.5:
            return rounded_value + 0.25
        else:
            return rounded_value
    
    def get_score(self, i, j):
        dist = self.distance[self.frozen["pos_id"], j]

        # Cas du résidu bloqué avec sa propre position
        if (i == self.frozen["seq_id"] or j == self.frozen["pos_id"]):
            return 0
        
        closest_dist = self.round_distance(dist)

        score = self.dope.loc[(self.dope['res1'] == self.aa_codes[self.frozen['seq_res']]) & 
                              (self.dope['res2'] == self.aa_codes[self.sequence[i]]), 
                              closest_dist]
        
        return float(score.values[0])
    
    def fill_matrix(self):
        # Partie supérieure gauche
        self.initialize_matrix(self.get_score(0, 0), [0, 0], 
                               [self.frozen['seq_id'], self.frozen['pos_id']],
                               self.get_score)

        for i in range(1, self.frozen['seq_id'] + 1):
            for j in range(1, self.frozen['pos_id'] + 1):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )
        # Partie inférieure droite
        self.initialize_matrix(self.matrix[self.frozen['seq_id'], self.frozen['pos_id']],
                               [self.frozen['seq_id'], self.frozen['pos_id']],
                               [self.lines - 1, self.columns - 1],
                               self.get_score)

        for i in range(self.frozen['seq_id'] + 1, self.lines):
            for j in range(self.frozen['pos_id'] + 1, self.columns):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )

        max_score = self.matrix[self.lines - 1, self.columns - 1]
        return max_score

## Nouveau code

In [2]:
# Matrice de distance
PDB_FILE = "../data/5awl.pdb"
TEMPLATE = Template(PDB_FILE)
DIST_MATRIX = TEMPLATE.build_dist_matrix()
    
# Matrice DOPE
DOPE_FILE = "../data/dope.par"
DOPE_MATRIX = clean_DOPE_data(DOPE_FILE)
    
FROZEN = {'seq_id': 9, 'pos_id': 0}
SEQUENCE = "YYDPETGTWY"
GAP = 0
    
LOW_TEST = LowLevelMatrix(GAP, FROZEN, DIST_MATRIX, DOPE_MATRIX, SEQUENCE)
MAX_SCORE = LOW_TEST.fill_matrix()
print(MAX_SCORE)
print(LOW_TEST.matrix)

0.0
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [3]:
class HighLevelMatrix(DynamicMatrix):
    def __init__(self, gap, distance, dope, sequence):
        lines = len(sequence)
        columns = len(distance)

        DynamicMatrix.__init__(self, lines, columns, gap)

        self.distance = distance
        self.dope = dope
        self.sequence = sequence

        self.get_score_matrix()

    def get_score_matrix(self):
        self.score_matrix = np.zeros((self.lines, self.columns))
        for i in range(self.lines):
            for j in range(self.columns):
                frozen = {'seq_id': i, 'pos_id': j}
                low_level = LowLevelMatrix(self.gap, frozen, self.distance, self.dope, self.sequence)
                self.score_matrix[i, j] =  low_level.fill_matrix()

    def get_score(self, i, j):
        score = self.score_matrix[i, j]
        return score

    def fill_matrix(self):
        # Initialisation
        self.initialize_matrix(self.get_score(0, 0), [0, 0], 
                               [self.lines - 1, self.columns - 1],
                               self.get_score)

        print(self.matrix)
        
        # Remplissage
        for i in range(1, self.lines):
            for j in range(1, self.columns):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )
        max_score = self.matrix[self.lines - 1, self.columns - 1]

        return max_score

    def get_alignment(self):
        structure_align = []
        sequence_align = []
        
        i = self.lines - 1
        j = self.columns - 1
        while not ((i == 0) and (j == 0)):
            print(i, j)
            square = self.matrix[i, j]
            score = self.score_matrix[i, j]
            # Match
            if (square == self.matrix[i - 1, j - 1] + score):
                print("match")
                structure_align.insert(0, j + 1)
                sequence_align.insert(0, self.sequence[i])
                i = i - 1
                j = j - 1
            # Gap
            else:
                if (square == self.matrix[i - 1, j] + score + self.gap):
                    print("gap structure")
                    structure_align.insert(0, '-')
                    sequence_align.insert(0, self.sequence[i])
                    i = i - 1
                elif (square == self.matrix[i, j - 1] + score + self.gap):
                    print("gap sequence")
                    structure_align.insert(0, j + 1)
                    sequence_align.insert(0, '-')
                    j = j - 1

        return ''.join(sequence_align), ''.join(str(x) for x in structure_align)

In [4]:
FROZEN = {'seq_id': 1, 'pos_id': 9}
SEQUENCE = "YYDPETGTWY"
GAP = 0
    
LOW_TEST = LowLevelMatrix(GAP, FROZEN, DIST_MATRIX, DOPE_MATRIX, SEQUENCE)
MAX_SCORE = LOW_TEST.fill_matrix()
print(MAX_SCORE)
print(LOW_TEST.matrix)

-2.54
[[-0.52 -0.76 -0.43 -0.58 -0.56 -0.86 -1.09 -1.13 -2.54 -2.54]
 [-0.52 -0.76 -0.76 -0.76 -0.76 -0.86 -1.09 -1.13 -2.54 -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -2.54]]


In [21]:
# Matrice de distance
PDB_FILE = "../data/5awl.pdb"
TEMPLATE = Template(PDB_FILE)
DIST_MATRIX = TEMPLATE.build_dist_matrix()

# Matrice DOPE
DOPE_FILE = "../data/dope.par"
DOPE_MATRIX = clean_DOPE_data(DOPE_FILE)
    
FROZEN = {'seq_id': 5, 'pos_id': 5}
SEQUENCE = "YYDPETGTWY"
GAP = 0

HIGH_TEST = HighLevelMatrix(GAP, DIST_MATRIX, DOPE_MATRIX, SEQUENCE)
print(HIGH_TEST.score_matrix)

[[-14.84 -15.14 -15.91 -14.81 -14.   -14.61 -14.08 -13.85 -13.81   0.  ]
 [-13.43 -15.14 -15.95 -15.07 -14.48 -15.3  -16.01 -15.22 -14.67  -2.54]
 [-12.55 -15.41 -16.61 -16.05 -16.16 -16.4  -16.92 -16.29 -15.    -3.74]
 [-11.31 -16.11 -16.96 -16.74 -16.39 -16.69 -17.04 -16.4  -15.84  -6.61]
 [ -8.54 -15.76 -16.4  -15.63 -15.87 -16.1  -16.59 -16.04 -14.94  -6.93]
 [ -7.11 -15.24 -16.1  -15.54 -15.29 -15.62 -16.33 -15.49 -14.82  -9.05]
 [ -5.33 -15.45 -16.06 -15.27 -15.   -15.47 -16.12 -15.57 -14.87 -10.19]
 [ -3.89 -15.24 -16.1  -15.54 -15.29 -15.62 -16.33 -15.49 -14.82 -12.27]
 [ -2.51 -14.6  -15.31 -14.72 -14.58 -14.8  -15.47 -14.75 -14.17 -12.97]
 [  0.   -13.81 -13.85 -14.07 -14.29 -14.5  -15.74 -15.18 -14.67 -14.94]]


In [6]:
print(HIGH_TEST.matrix)
max_score = HIGH_TEST.fill_matrix()
print(HIGH_TEST.matrix)
print(max_score)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ -16.25  -34.21  -52.98  -70.87  -88.17 -106.29 -125.12 -143.16 -160.65
  -163.19]
 [ -31.09    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ -45.26    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ -58.03    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ -68.51    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ -77.17    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ -84.19    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ -89.63    0.      0.      0.      0.      0.      0.      0.      0.
     0.  ]
 [ 

In [7]:
print(HIGH_TEST.matrix[9, 9])
print(HIGH_TEST.matrix[8, 8])
print(HIGH_TEST.score_matrix[9, 9])
print(HIGH_TEST.matrix[9, 8] + HIGH_TEST.score_matrix[9, 9])

-358.37
-322.86999999999995
-16.35
-358.37


In [8]:
seq, struct = HIGH_TEST.get_alignment()
print(seq, struct)

9 9
gap sequence
9 8
gap sequence
9 7
gap sequence
9 6
gap structure
8 6
gap structure
7 6
gap structure
6 6
gap structure
5 6
gap structure
4 6
gap sequence
4 5
gap sequence
4 4
gap sequence
4 3
gap sequence
4 2
gap structure
3 2
gap structure
2 2
gap structure
1 2
gap structure
0 2
gap sequence
0 1
gap sequence
--YDPE----TGTWY--- 23----4567-----8910


In [9]:
print(seq)
print(struct)

--YDPE----TGTWY---
23----4567-----8910


In [10]:
print(HIGH_TEST.sequence)

YYDPETGTWY


In [11]:
print(DOPE_MATRIX)

    res1 res2   0.25   0.75   1.25   1.75   2.25   2.75   3.25   3.75  ...  \
0    ALA  ALA  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.64  ...   
1    ALA  ARG  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.61  ...   
2    ALA  ASN  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.58  ...   
3    ALA  ASP  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.62  ...   
4    ALA  CYS  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.49  ...   
..   ...  ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
395  VAL  SER  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.48  ...   
396  VAL  THR  10.00  10.00  10.00  10.00  10.00   5.84   6.15  -1.56  ...   
397  VAL  TRP  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.27  ...   
398  VAL  TYR  10.00  10.00  10.00  10.00  10.00  10.00   5.71  -1.38  ...   
399  VAL  VAL  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.48  ...   

     10.25  10.75  11.25  11.75  12.25  12.75  13.25  13.75  14

In [12]:
DOPE_MATRIX.to_csv('matrix.csv', index=False)

In [13]:
i = 0
j = 0
print((i >= 0) and (j >= 0))
print(not ((i == 0) and (j == 0)))

True
False


In [14]:
import pandas as pd
import numpy as np

class CarboneAlpha:
    """Classe pour représenter un carbone alpha d'une protéine."""
    
    def __init__(self, number, x, y, z):
        self.number = number
        self.x = x
        self.y = y
        self.z = z
    
    def compute_distance(self, other):
        dist = ((other.x - self.x) ** 2 + (other.y - self.y) ** 2 + (other.z - self.z) ** 2) ** 0.5
        return dist

class Template:
    """Classe pour représenter le template utilisé."""

    def __init__(self, file):
        self.structure = self.build_template_from_pdb(file)
        self.length = len(self.structure)

    def build_template_from_pdb(self, filename):
        list_calpha = []
        with open(filename, "r") as pdb :
            for ligne in pdb:
                if ligne.startswith("ATOM") and (ligne[12:16].strip() == "CA"):
                    number = ligne[6:11].strip()
                    x = float(ligne[30:38].strip())
                    y = float(ligne[38:46].strip())
                    z = float(ligne[46:54].strip())
                                       
                    list_calpha.append(CarboneAlpha(number, x, y, z))
        return list_calpha
        
    def build_dist_matrix(self):
        dist_list = []
        
        for i, atom in enumerate(self.structure):
            dist_ligne = []
            for other in (self.structure):
                dist_ligne.append(atom.compute_distance(other))
            dist_list.append(dist_ligne)
            
        dist_matrix = np.array(dist_list)
        return dist_matrix
    
    def __str__(self):
        string = ""
        for i, ca in enumerate(self.structure):
            string += f"position {i}-{ca.number}, coor( {ca.x}, {ca.y}, {ca.z})\n"
        return string

def clean_DOPE_data(filename):
    ca_matrix = []
    
    with open(filename, "r") as dope :
        for ligne in dope:
            if ligne[3:7].strip() == "CA" and ligne[11:14].strip() == "CA":
                ca_matrix.append(ligne.split())
    
    columns = ['res1', 'temp1', 'res2', 'temp2'] + list(np.arange(0.25, 15, 0.5))
    dope_score = pd.DataFrame(ca_matrix, columns = columns) 
    dope_score = dope_score.drop(['temp1', 'temp2'], axis=1)
    
    return dope_score

def get_fasta_sequence(filename):
    sequence = ""
    with open(filename, "r") as fasta:
        for ligne in fasta:
            if ligne.startswith(">"):
                continue
            sequence += ligne.strip()
    return sequence

class DynamicMatrix:    
    def __init__(self, lines, columns, gap):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.gap = gap

    def initialize_matrix(self, first_val, start, end, get_score):
        if (start[0] < 0) or (start[1] < 0):
            raise ValueError("Start of initialization out of matrix.")
        if (end[0] >= self.lines) or (end[1] >= self.columns):
            raise ValueError("End of initialization out of matrix.")
        
        # Première case
        self.matrix[start[0], start[1]] = first_val
        
        # Remplissage de la première colonne jusqu'à la limite
        for i in range(start[0] + 1, end[0] + 1):
            self.matrix[i, start[1]] = self.matrix[i - 1, start[1]] + self.gap + get_score(i, start[1])

        # Remplissage de la première ligne jusqu'à la limite
        for j in range(start[1] + 1, end[1] + 1):
            self.matrix[start[0], j] = self.matrix[start[0], j - 1] + self.gap + get_score(start[0], j)

class LowLevelMatrix(DynamicMatrix):
    aa_codes = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
    }
    
    def __init__(self, gap, frozen, distance, dope, sequence):
        lines = len(sequence)
        columns = len(distance)
        
        DynamicMatrix.__init__(self, lines, columns, gap)

        # Vérification du blocage de la case
        if (frozen['seq_id'] >= lines) or (frozen['seq_id'] < 0):
            raise ValueError("Frozen line index out of matrix.")
        if (frozen['pos_id'] >= columns) or (frozen['pos_id'] < 0):
            raise ValueError("Frozen column index out of matrix")

        # Récupération du résidu fixé
        frozen['seq_res'] = sequence[frozen['seq_id']]
        
        self.frozen = frozen
        self.distance = distance
        self.dope = dope
        self.sequence = sequence

    def round_distance(self, dist):
        # arrondi au quart le plus proche
        rounded_value = round(dist * 4) / 4
        
        # ne garde que 0.25 ou 0.75
        decimal = rounded_value % 1
        if decimal == 0.0:
            return rounded_value + 0.25
        elif decimal == 0.5:
            return rounded_value + 0.25
        else:
            return rounded_value
    
    def get_score(self, i, j):
        dist = self.distance[self.frozen["pos_id"], j]
        closest_dist = self.round_distance(dist)

        score = self.dope.loc[(self.dope['res1'] == self.aa_codes[self.frozen['seq_res']]) & 
                              (self.dope['res2'] == self.aa_codes[self.sequence[i]]), 
                              closest_dist]
        
        return float(score.values[0])
    
    def fill_matrix(self):
        # Partie supérieure gauche
        self.initialize_matrix(self.get_score(0, 0), [0, 0], 
                               [self.frozen['seq_id'], self.frozen['pos_id']],
                               self.get_score)

        for i in range(1, self.frozen['seq_id'] + 1):
            for j in range(1, self.frozen['pos_id'] + 1):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )
        # Partie inférieure droite
        self.initialize_matrix(self.matrix[self.frozen['seq_id'], self.frozen['pos_id']],
                               [self.frozen['seq_id'], self.frozen['pos_id']],
                               [self.lines - 1, self.columns - 1],
                               self.get_score)

        for i in range(self.frozen['seq_id'] + 1, self.lines):
            for j in range(self.frozen['pos_id'] + 1, self.columns):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )

        max_score = self.matrix[self.lines - 1, self.columns - 1]
        return max_score

class HighLevelMatrix(DynamicMatrix):
    def __init__(self, gap, query, template, dope):
        distance = template.build_dist_matrix()
        lines = len(query)
        columns = len(distance)

        DynamicMatrix.__init__(self, lines, columns, gap)

        self.sequence = query
        self.distance = distance
        self.dope = dope

        self.get_score_matrix()

    def get_score_matrix(self):
        self.score_matrix = np.zeros((self.lines, self.columns))
        for i in range(self.lines):
            for j in range(self.columns):
                frozen = {'seq_id': i, 'pos_id': j}
                low_level = LowLevelMatrix(self.gap, frozen, self.distance, self.dope, self.sequence)
                self.score_matrix[i, j] =  low_level.fill_matrix()

    def get_score(self, i, j):
        score = self.score_matrix[i, j]
        return score

    def fill_matrix(self):
        # Initialisation
        self.initialize_matrix(self.get_score(0, 0), [0, 0], 
                               [self.lines - 1, self.columns - 1],
                               self.get_score)
        
        # Remplissage
        for i in range(1, self.lines):
            for j in range(1, self.columns):
                score = self.get_score(i, j)
                self.matrix[i, j] = score + min(self.matrix[i - 1, j - 1],
                                                self.matrix[i - 1, j] + self.gap,
                                                self.matrix[i, j - 1] + self.gap
                                               )
        max_score = self.matrix[self.lines - 1, self.columns - 1]

        return max_score

    def get_alignment(self):
        structure_align = []
        sequence_align = []
        
        i = self.lines - 1
        j = self.columns - 1
        while not ((i == 0) and (j == 0)):
            print(i, j)
            square = self.matrix[i, j]
            score = self.score_matrix[i, j]
            # Match
            if (square == self.matrix[i - 1, j - 1] + score):
                print("match")
                structure_align.insert(0, j + 1)
                sequence_align.insert(0, self.sequence[i])
                i = i - 1
                j = j - 1
            # Gap
            else:
                if (square == self.matrix[i - 1, j] + score + self.gap):
                    print("gap structure")
                    structure_align.insert(0, '-')
                    sequence_align.insert(0, self.sequence[i])
                    i = i - 1
                elif (square == self.matrix[i, j - 1] + score + self.gap):
                    print("gap sequence")
                    structure_align.insert(0, j + 1)
                    sequence_align.insert(0, '-')
                    j = j - 1

        return ''.join(sequence_align), ''.join(str(x) for x in structure_align)

if __name__ == "__main__":
    # Séquence 'query'
    FASTA_FILE = "../data/5AWL.fasta"
    QUERY = get_fasta_sequence(FASTA_FILE)

    # Structure 'template'
    PDB_FILE = "../data/5awl.pdb"
    TEMPLATE = Template(PDB_FILE)
    
    # Matrice de distance
    #DIST_MATRIX = TEMPLATE.build_dist_matrix()
    
    # Matrice DOPE
    DOPE_FILE = "../data/dope.par"
    DOPE_MATRIX = clean_DOPE_data(DOPE_FILE)

    # Information(s) supplémentaire(s)
    GAP = 0

    # Algorithme principal
    HIGH_LEVEL = HighLevelMatrix(GAP, QUERY, TEMPLATE, DOPE_MATRIX)
    MAX_SCORE = HIGH_LEVEL.fill_matrix()
    print(MAX_SCORE)
    ALIGN_SEQ, ALIGN_STRUCT = HIGH_LEVEL.get_alignment()
    print(ALIGN_SEQ)
    print(ALIGN_STRUCT)

-168.37
9 9
gap sequence
9 8
gap sequence
9 7
gap sequence
9 6
gap structure
8 6
gap structure
7 6
gap structure
6 6
gap structure
5 6
gap structure
4 6
gap sequence
4 5
gap sequence
4 4
gap sequence
4 3
gap sequence
4 2
gap structure
3 2
gap structure
2 2
gap structure
1 2
gap structure
0 2
gap sequence
0 1
gap sequence
--YDPE----TGTWY---
23----4567-----8910
